In [1]:
import snap
import random
import numpy
S = 10**4
TPATH = "/courses/TSKS33/ht2023/data/"
def genmod10star():   
    #G1 = snap.GenPrefAttach(1000, 10)
    #G1 = snap.GenRndPowerLaw(100000,2.2)
    G = snap.GenStar(snap.PUNGraph,10,False)
    G.AddEdge(4,5)
    #G = snap.GetMxScc(G1)
    N = G.GetNodes()
    
    # node numbering in the mat file is sequential n=1,2,... following the node iterator
    # gio.h line 369
#    snap.SaveMatlabSparseMtx(G, "mod10star.mat")

    # assign the degree as attribute
    x = snap.TIntFltH(N)
#    file = open("mod10star_attr.mat",'w') 
    for NI in G.Nodes():
        n=NI.GetId()
        x[n]=NI.GetDeg()
#        file.write(str(n+1)+"\t"+str(x[n])+"\n")
#        file.write(str(x[n])+"\n")
#    file.close()
    
    return G, x

def genLiveJournal():   
    G1 = snap.LoadEdgeList(snap.PUNGraph, TPATH + "soc-LiveJournal1.txt", 0, 1)
    snap.DelSelfEdges(G1)
    G = snap.GetMxScc(G1)
#    print(snap.IsConnected(G))
    N = G.GetNodes()

#    snap.SaveMatlabSparseMtx(G, "LiveJournal.mat")

    # assign synthetic data as attributes
    x = snap.TIntFltH(N)
#    file = open("LiveJournal_attr.mat",'w') 
    for NI in G.Nodes():
        n=NI.GetId()
        k=NI.GetDeg()
        x[n]=50000*(1+k/10.+0.1*numpy.sin(n))
#        file.write(str(n+1)+"\t"+str(x[n])+"\n")
#        file.write(str(x[n])+"\n")
 
#    file.close()
    return G, x


In [2]:
G, x = genLiveJournal()

In [3]:

sum = 0
for n in G.Nodes():
    t = x[n.GetId()]
    sum+=t
exact = sum/G.GetNodes()
print("-- expected values of <x>-hat -----")
print("uniform sampling:", exact)

sum = 0
for i in G.Nodes():
    node = G.GetNI(i.GetId())
    tmp = 0
    for nbr_id in node.GetOutEdges():
        nbr = G.GetNI(nbr_id)
        t = x[nbr.GetId()]
        tmp+=t
    sum += (tmp / node.GetDeg())

print("random connection of random node:", round(sum/G.GetNodes(), 3))

sum = 0

for node in G.Nodes():
    for nbr_id in node.GetOutEdges():
        nbr = G.GetNI(nbr_id)
        sum += x[nbr.GetId()]
    
print("uniform random walk:", sum/(2*G.GetEdges()))

sum = 0
for n in G.Nodes():
    t = x[n.GetId()]
    sum+=t
print("M-H random walk:", sum/G.GetNodes())

-- expected values of <x>-hat -----
uniform sampling: 138451.894391101
random connection of random node: 598522.277
uniform random walk: 903319.3950024237
M-H random walk: 138451.894391101


In [8]:
num_runs = 5
for i in range(num_runs):
    tmp_sum = 0
    Rnd = snap.TRnd(i)
    Rnd.Randomize()
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        tmp_sum += x[nid]
    print("uniform sampling:", tmp_sum/S)


uniform sampling: 139571.70862823448
uniform sampling: 137391.55129453787
uniform sampling: 136302.2363130672
uniform sampling: 140228.45370975218
uniform sampling: 140897.03669778034


In [9]:
import random
for i in range(num_runs):
    sum = 0
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        node = G.GetNI(nid)
        e = random.randint(0, node.GetOutDeg()-1)
        nbr = node.GetOutNId(e)
        #nbr = G.GetNI(nbr)
        sum+=x[nbr]

    print("random connection of random node:", sum/S)

random connection of random node: 581940.7437629229
random connection of random node: 608974.2675531083
random connection of random node: 563859.3260152295
random connection of random node: 593283.6973837701
random connection of random node: 579089.280687454


In [10]:
for i in range(num_runs):
    sum = 0
    node = G.GetNI(0)
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        node = G.GetNI(nid)
    for i in range(S):
        e = random.randint(0, node.GetOutDeg()-1)
        nbr = node.GetOutNId(e)
        node = G.GetNI(nbr)
        sum += x[nbr]
    
    print("uniform random walk:", sum/S)

uniform random walk: 941757.5440663754
uniform random walk: 873539.5150545037
uniform random walk: 770676.1302613303
uniform random walk: 802471.8977561916
uniform random walk: 842939.0188768967


In [12]:
for i in range(num_runs):
    sum = 0
    node = G.GetNI(0)
    for i in range(S):
        nid = G.GetRndNId(Rnd)
        node = G.GetNI(nid)

    for i in range(S):
        e = random.randint(0, node.GetOutDeg()-1)
        nbr = node.GetOutNId(e)
        nbr = G.GetNI(nbr)
        nbr_deg = nbr.GetDeg()
        node_deg = node.GetDeg()
        sum += x[node.GetId()]
        if random.uniform(0, 1) < (node_deg/nbr_deg):
            node = nbr
    print("M-H random walk:", sum/S)

M-H random walk: 125860.21529546317
M-H random walk: 138447.0201811939
M-H random walk: 158040.30810900117
M-H random walk: 132008.9891004763
M-H random walk: 124439.31907512297
